In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # visualization library

#warnings.filterwarnings('ignore')
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

## Data Loading

In [ ]:
TRAIN_PATH = os.path.join("..", "input", "train.csv")
TEST_PATH = os.path.join("..", "input", "test.csv")

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

print(f"training set shape : {train.shape}")
print(f"testing set shape : {test.shape}")

In [ ]:
train.head()

In [ ]:
train.info()

## Data Exploration

In [ ]:
train.isna().sum()

In [ ]:
train.trip_duration.min()

In [ ]:
train.trip_duration.max()

 Pour pas fausser les résultats on décide d'enlever les trajets ou il y a un temps minimum d'une seconde et un maximum de 40jours.
Je vais estimer qu'un temps de trajet doit être compris entre 1-5min et plusieurs heures

## Data Preprocessing


Gestion des outliers

In [ ]:
train.describe()

Il y a des valeurs aberrantes pour les trip_duration en min et max. 

In [ ]:
plt.subplots(figsize=(18,6))
plt.title("Visualisation des outliers")
train.boxplot();

## Features engineering

In [ ]:
col_diff = list(set(train.columns).difference(set(test.columns)))
col_diff

In [ ]:
#Calcule de la distance entre drop et pickup
train['dist'] = np.sqrt((train['pickup_latitude']-train['dropoff_latitude'])**2
                        + (train['pickup_longitude']-train['dropoff_longitude'])**2)
test['dist'] = np.sqrt((test['pickup_latitude']-test['dropoff_latitude'])**2
                        + (test['pickup_longitude']-test['dropoff_longitude'])**2)

In [ ]:
#il n'est pas nécessaire d'avoir 0 passager, nous allons les enlever
train = train[train['passenger_count']>= 1]

In [ ]:
# La durée du voyage est comprise entre 1 sec. et 3526282 sec.
# Nous laisserons tomber les valeurs inférieures à 1 min 30 (90sec) et supérieures à 166 min (10 000 sec).
train = train[train['trip_duration']>= 1.5 ]
train = train[train['trip_duration']<= 10000 ]

In [ ]:
# Nous allons laisser tomber la longitude et la latitude (On drop ce qui ressemble à des valeurs aberrantes)
train = train.loc[train['pickup_longitude']> -90]
train = train.loc[train['pickup_latitude']< 47.5]

train = train.loc[train['dropoff_longitude']> -90]
train = train.loc[train['dropoff_latitude']> 34]

In [ ]:
#convertir le string en datetime pour avoir que l'heure 
train['pickup_datetime'] = pd.to_datetime(train['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
test['pickup_datetime'] = pd.to_datetime(test['pickup_datetime'], format='%Y-%m-%d %H:%M:%S')

In [ ]:
#ajout dans une nouvelle colonne
train['hour'] = train.loc[:,'pickup_datetime'].dt.hour;
test['hour'] = test.loc[:,'pickup_datetime'].dt.hour;

In [ ]:
#Ajout dans une nouvelle colonne à train 
X_train = train[["passenger_count","vendor_id", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude", "dist", "hour" ]]


In [ ]:
y_train = train["trip_duration"]  # This is our target
X_train = train[["passenger_count","vendor_id", "pickup_longitude", "pickup_latitude", "dropoff_longitude","dropoff_latitude", "dist","hour" ]]

## Selection du modèle


In [ ]:
# importer la lib pour cross valider le model
from sklearn.model_selection import cross_val_score

# importer la lib pour la regression de Random Forest
from sklearn.ensemble import RandomForestRegressor

# importer la lib pour la regression de Random Forest
from sklearn.linear_model import SGDRegressor

from sklearn.linear_model import LinearRegression

from sklearn.model_selection import ShuffleSplit

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, random_state=42)


In [ ]:
sgd = SGDRegressor()
sgd.fit(X_train, y_train)

## 6. Entrainement du ou des modèle(s) & Predictions¶


In [ ]:
b2o = RandomForestRegressor(n_estimators=19, min_samples_split=2, min_samples_leaf=4, max_features='auto', max_depth=80, bootstrap=True)
b2o.fit(X_train, y_train)

In [ ]:
cv = ShuffleSplit(n_splits=4, test_size=0.8, random_state=42)
cv_scores = cross_val_score(b2o, X_train, y_train, cv=cv, scoring= 'neg_mean_squared_log_error')

In [ ]:
cv_scores

In [ ]:
for i in range(len(cv_scores)):
    cv_scores[i] = np.sqrt(abs(cv_scores[i]))
print(np.mean(cv_scores))

In [ ]:
## Prediction

In [ ]:
test.head()

In [ ]:
X_fit = test[["vendor_id", "passenger_count","pickup_longitude", "pickup_latitude","dropoff_longitude","dropoff_latitude","dist","hour"]]
prediction = b2o.predict(X_fit)
prediction

In [ ]:
my_submission = pd.DataFrame({'id': test.id, 'trip_duration': prediction})
my_submission.head()

In [ ]:
my_submission.to_csv('submission.csv', index=False)